In [0]:
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-cdp-bu-finance-dev`.curated.EPMParentChildHierarchy
  WHERE
    `IsBaseRecord` = 1
    AND adjacencypath like 'Entity|Shared_Mbr_Entity_Structure|Management|BU_NA|%'
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
extract as (
  SELECT
    TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
    bn.alias as Entity,
    CASE
      WHEN CostType = 'COS DIRECT' THEN 'DIRECT USD'
      WHEN CostType = 'COS INDIRECT' THEN 'INDIRECT USD'
      WHEN CostType = 'G&A' THEN 'INDIRECT USD'
      WHEN CostType = 'SELLING' THEN 'SELLING USD'
    END as Account,
    CASE
      WHEN Breakdown2Id = 'TKE101' THEN 'NI'
      WHEN Breakdown2Id = 'TKE102' THEN 'MOD'
      WHEN Breakdown2Id = 'TKE1031' THEN 'SERV'
      WHEN Breakdown2Id = 'TKE1032' THEN 'REPAIR'
    END AS LOB,
    AmountAnnual as Value
  FROM
    `na-cdp-bu-hr-dev`.curated.usdamount as a
      INNER JOIN onlybaseentities as b
        on b.child = a.EPMBranch
      LEFT JOIN branchnames as bn
        on bn.backendname = a.EPMBranch
  WHERE
    a.DateInt >= 20231001
    and a.AmountAnnual <> 0
    and a.CostType IN ('COS DIRECT', 'COS INDIRECT', 'G&A', 'SELLING')
    and a.Breakdown2Id in ('TKE101', 'TKE102', 'TKE1031', 'TKE1032')
),
resultsgrouped as (
  SELECT
    `Calendar Date`,
    Account,
    LOB,
    Entity,
    ROUND(SUM(Value), 3) as Value,
    grouping(Entity) as grp_Entity
  FROM
    extract
  GROUP BY
    GROUPING SETS ((`Calendar Date`, LOB, Account, Entity), (`Calendar Date`, LOB, Account))
)
SELECT
  *,
  'Actuals' as Scenario,
  'Headcount USD' as Source
FROM
  (
    SELECT
      `Calendar Date`,
      Account,
      LOB,
      Entity,
      sum(Value) as Value
    FROM
      resultsgrouped
    WHERE
      grp_Entity = 0
    GROUP BY
      `Calendar Date`,
      Account,
      LOB,
      Entity
    UNION ALL
    SELECT
      `Calendar Date`,
      Account,
      LOB,
      'BU North America' as Entity,
      sum(Value) as Value
    FROM
      resultsgrouped
    WHERE
      grp_Entity = 1
    GROUP BY
      `Calendar Date`,
      Account,
      LOB
  )